In [1]:
import search_engine.util as util
import re
import os
import pandas
import re
import sys
import copy

In [2]:
WIKI_DUMP_FILENAME = 'data/frwiki-20151226-pages-articles.xml'
WIKI_DIR = 'data/wiki'
WIKI_N_LINES = 246871655

In [3]:
if not os.path.exists(WIKI_DIR):
    os.mkdir(WIKI_DIR)

Functions to print progress during file browsing

In [4]:
def pretty_number(n):
    a = str(n)[::-1]
    return (' '.join([a[i:i+3] for i in range(0, len(a), 3)]))[::-1]

def print_progress(current_line):
    global last_line, last_print, WIKI_N_LINES
    if current_line - last_line < 100000:
        return
    sys.stdout.write('\b' * len(last_print))
    sys.stdout.write('\r')
    s = '{} / {}'.format(pretty_number(current_line), pretty_number(WIKI_N_LINES))
    sys.stdout.write(s)
    sys.stdout.flush()
    last_line = current_line
    last_print = s

## Load words dictionnary

In [5]:
dataframe = pandas.read_csv('data/dictionnary.fr.csv')
word_to_id = {}
id_to_word = {}
for i in range(len(dataframe)):
    word_to_id[dataframe['word'][i]] = dataframe['id'][i]
    id_to_word[dataframe['id'][i]] = dataframe['word'][i]

## Load page_to_id dict

In [8]:
page_to_id = {}
with open(os.path.join(WIKI_DIR, 'page_to_id.csv'), 'r') as fd:
    for line in fd.readlines()[1:]:
        line_s = line[:-1].split('@')
        page_to_id[line_s[0]] = sorted([int(d) for d in line_s[1].split(' ')])

ValueError: invalid literal for int() with base 10: '2.-other-logo.png'

## Load id_to_page dict

In [ ]:
id_to_page = {}
with open(os.path.join(WIKI_DIR, 'id_to_page.csv'), 'r') as fd:
    for line in fd.readlines()[1:]:
        line_s = line[:-1].split('@')
        id_to_page[int(line_s[0])] = line_s[1]

## Function: extract links from page content

In [ ]:
def extract_links(page_id, words):
    global page_to_id
    re_link = '\[\[(.*?)\]\]'
    match = re.findall(re_link, words)
    links = []
    if match:
        for m in match:
            for m1 in m.split('|'):
                m1 = util.normalize_page_content(m1)
                if m1 in page_to_id:
                    links = links + page_to_id[m1]
    return sorted(set(links) - set([page_id]))

## Function: extract words id from page content

In [22]:
def extract_words_id(words):
    global word_to_id
    words = util.normalize_page_content(words)
    words_id = [word_to_id[w] for w in words.split(' ') if w in word_to_id]
    return words_id

In [23]:
fd_wiki_dump = open(WIKI_DUMP_FILENAME, 'r')

in_page = False
in_text = False
words = None
page_title = None

re_id = '.*<id>(\d+)</id>.*'
re_text_start = '^.*<text.*>(.+)$'
re_text_end = '^(.*)</text>'

last_print = ''
last_line = 0
line_count = 0

fd_links = open(os.path.join(WIKI_DIR, 'page_links.csv'), 'w')
fd_links.write('page_id,links\n')

#fd_words = open(os.path.join(WIKI_DIR, 'page_words.csv'), 'w')
#fd_words.write('page_id,words\n')

page_to_id_cpy = copy.deepcopy(page_to_id)

for line in fd_wiki_dump:
    line = line[:-1]
    line = line.strip()
    line = util.process_word(line.decode('utf-8'))
    if in_page:
        if in_text:
            if '</text' in line:
                match = re.match(re_text_end, line)
                if match is None:
                    continue
                words += ' ' + match.group(1)
                in_text = False
            else:
                words += ' ' + line
        else:
            if '</page' in line:
                if page_title and words:
                    page_id = page_to_id_cpy[page_title][0]
                    page_to_id_cpy[page_title] = page_to_id_cpy[page_title][1:]
                    #links, words_id = extract_links_and_words_id(page_id, words)
                    links = extract_links_and_words_id(page_id, words)
                    fd_links.write('{},{}\n'.format(page_id, ' '.join([str(k) for k in links])))
                    #fd_words.write('{},{}\n'.format(page_id, ' '.join([str(k) for k in words_id])))
                words = None
                page_title = None
                in_page = False
            elif '<title' in line:
                match = re.match(re_title, line)
                if match is None:
                    continue
                page_title = match.group(1)
            elif '<text' in line:
                match = re.match(re_text_start, line)
                if match is None:
                    continue
                words = match.group(1)
                in_text = True
    else:
        if '<page' in line:
            in_page = True
    print_progress(line_count)
    line_count += 1

#fd_links.close()
fd_words.close()
fd_wiki_dump.close()

246 800 000 / 246 871 655